In [ ]:
import numpy as np
from dataclasses import dataclass
from matplotlib import pyplot as plt

def dot(a, b):
    return np.multiply(a, b).sum(axis=-2)

def length(v):
    return np.linalg.norm(v, axis=-2)

def normalize(v):
    return v / np.sqrt(np.multiply(v, v).sum(axis=-2, keepdims=True))

def cross(a, b):
    return np.cross(a, b, axis=-2)

def make_vec3(x, y, z):
    return np.array((x, y, z)).reshape(3, 1)

def reflect(V, N):
    return 2 * dot(N, V) * N - V

def polar_to_r3(phi, axis=0):
    phi = phi if len(np.shape(phi)) > 0 else np.reshape(phi, 1)
    p = np.stack((np.cos(phi), np.zeros(np.shape(phi)), np.sin(phi)), axis=axis)
    return p

def spherical_to_r3(phi, theta, axis=0):
    z = np.cos(theta)
    r = np.sqrt(1 - z * z)
    return np.stack((np.cos(phi) * r, np.sin(phi) * r, z), axis=axis)

In [ ]:
from pathlib import Path
import subprocess
import ctypes
from numpy.ctypeslib import ndpointer

CMAKE_PATH = "cmake"
PROJECT_DIR = Path.cwd().parent
BUILD_DIR = PROJECT_DIR.joinpath("build", "dev")

RENPY_TARGET = "renpy"
RENPY_LIB = f"lib{RENPY_TARGET}{"d" if BUILD_DIR.name == "debug" else ""}.so"
RENPY_LIB_PATH = BUILD_DIR.joinpath("bin", RENPY_LIB)

p = subprocess.run([CMAKE_PATH, "--build", BUILD_DIR, "-t", RENPY_TARGET], text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(p.stdout)
p.check_returncode()

librenpy = ctypes.CDLL(RENPY_LIB_PATH)
# extern "C" void ren_eval_brdf(size_t n, const float *cL, float *y, float f0, float roughness, float NoV);
librenpy.ren_eval_brdf.argtypes = [
    ctypes.c_size_t, # n
    ndpointer(ctypes.c_float, flags="F"), # L
    ndpointer(ctypes.c_float, flags=("F", "W")), # y
    ctypes.c_float, # f0
    ctypes.c_float, # roughness
    ctypes.c_float, # NoV
]
# extern "C" void ren_eval_analytical_sg_brdf(size_t n, const float *cL, float *y, float f0, float roughness, float NoV);
librenpy.ren_eval_analytical_sg_brdf.argtypes = [
    ctypes.c_size_t, # n
    ndpointer(ctypes.c_float, flags="F"), # L
    ndpointer(ctypes.c_float, flags=("F", "W")), # y
    ctypes.c_float, # f0
    ctypes.c_float, # roughness
    ctypes.c_float, # NoV
]
# extern "C" void ren_eval_sg_brdf(size_t n, const float *cL, float *y, float f0, float roughness, float NoV, size_t num_brdf_sgs);
librenpy.ren_eval_sg_brdf.argtypes = [
    ctypes.c_size_t, # n,
    ndpointer(ctypes.c_float, flags="F"), # L,
    ndpointer(ctypes.c_float, flags=("F", "W")), # y,
    ctypes.c_float, # f0,
    ctypes.c_float, # roughness,
    ctypes.c_float, # NoV,
    ctypes.c_size_t, # num_brdf_sgs
]

def eval_brdf(L, f0, roughness, NoV):
    assert(len(L.shape) == 2)
    assert(L.shape[0] == 3)
    if (L.dtype != np.float32 or not L.flags.f_contigious):
        L = np.array(L, dtype=np.float32, order='F')
    y = np.empty(L.shape[1], dtype=np.float32)
    librenpy.ren_eval_brdf(ctypes.c_size_t(y.size), L, y, ctypes.c_float(f0), ctypes.c_float(roughness), ctypes.c_float(NoV))
    return y
    
def eval_analytical_sg_brdf(L, f0, roughness, NoV):
    assert(len(L.shape) == 2)
    assert(L.shape[0] == 3)
    if (L.dtype != np.float32 or not L.flags.f_contigious):
        L = np.array(L, dtype=np.float32, order='F')
    y = np.empty(L.shape[1], dtype=np.float32)
    librenpy.ren_eval_analytical_sg_brdf(ctypes.c_size_t(y.size), L, y, ctypes.c_float(f0), ctypes.c_float(roughness), ctypes.c_float(NoV))
    return y

def eval_sg_brdf(L, f0, roughness, NoV, num_brdf_sgs):
    assert(len(L.shape) == 2)
    assert(L.shape[0] == 3)
    if (L.dtype != np.float32 or not L.flags.f_contigious):
        L = np.array(L, dtype=np.float32, order='F')
    y = np.empty(L.shape[1], dtype=np.float32)
    librenpy.ren_eval_sg_brdf(ctypes.c_size_t(y.size), L, y, ctypes.c_float(f0), ctypes.c_float(roughness), ctypes.c_float(NoV), ctypes.c_size_t(num_brdf_sgs))
    return y

In [ ]:
def loss(y, p):
    return np.sum(((y - p) / np.max(y))** 2)

for roughness in (0.1, 0.2, 0.3, 0.5, 7.5/8):
    for V_phi in (0, 45, 80, 88.5):
        NoV = np.cos(np.radians(V_phi))

        phi = np.linspace(-0.5 * np.pi, 1.5 * np.pi, num=1000)
        L = polar_to_r3(phi)

        for f0 in (0.04,):
            y = eval_brdf(L, f0, roughness, NoV)
            ap = eval_analytical_sg_brdf(L, f0, roughness, NoV)
    
            fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
            plt.title(f"F0={f0}, Roughness={roughness}, V_phi={V_phi:.1f}")
            ax.plot(phi, y, label="y")
            ax.plot(phi, ap, label=f"Analytical SG: {loss(y, ap):.2}")
            for i in range(1, 4 + 1):
                p = eval_sg_brdf(L, f0, roughness, NoV, i)
                ax.plot(phi, p, label=f"{i} SG(s): {loss(y, p):.2}")
            ax.legend()
            ax.grid(True)
            plt.show()
        
            fig, ax = plt.subplots()
            plt.title(f"F0={f0}, Roughness={roughness}, V_phi={V_phi:.1f}")
            ax.plot(np.pi / 2 - phi, y, label="y")
            ax.plot(np.pi / 2 - phi, ap, label=f"Analytical SG: {loss(y, ap):.2}")
            for i in range(1, 4 + 1):
                p = eval_sg_brdf(L, f0, roughness, NoV, i)
                ax.plot(np.pi / 2 - phi, p, label=f"{i} SG(s): {loss(y, p):.2}")
            ax.grid(True)
            ax.legend()
            plt.show()